In [ ]:
import os
import sys  
import json 
import copy
import torch
import argparse
import numpy as np
from PIL import Image 
from tqdm import tqdm 
from utils import model_gen, load_jsonl
from transformers import AutoModelForCausalLM, AutoTokenizer  
  
ckpt_path = 'internlm/internlm-xcomposer2-vl-7b'
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(ckpt_path, device_map="cuda", trust_remote_code=True).eval().cuda().half()
model.tokenizer = tokenizer

In [ ]:
meta_instruction = """You are an AI assistant whose name is InternLM-XComposer (浦语·灵笔).
- InternLM-XComposer (浦语·灵笔) is a multi-modality conversational language model that is developed by Shanghai AI Laboratory (上海人工智能实验室). It is designed to be helpful, honest, and harmless.
- InternLM-XComposer (浦语·灵笔) can understand and communicate fluently in the language chosen by the user such as English and 中文.
- InternLM-XComposer (浦语·灵笔) is capable of comprehending and articulating responses effectively based on the provided image."""

samples = json.load(open('data/mm-vet/mm-vet.json', 'r'))

count = 0
results = {}
for k, v in tqdm(samples.items()):
    im_path = 'data/mm-vet/images/'+v['imagename'] 
    text = '[UNUSED_TOKEN_146]system\n{}[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]user\n{}Answer this question in detail.[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]assistant\n'.format(meta_instruction, v['question'])
    with torch.cuda.amp.autocast(): 
        out = model_gen(model, text, im_path)
     
    results[k] = out


In [ ]:
json.dump(results, open('Output/MMVet_InternLM_XComposer_VL.json.json','w'), indent=2)